In [1]:
from pathlib import Path

import torch
import torch.nn.functional as F

/Users/william/Documents/src/makemore/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
# Create dataset

words = Path.cwd().joinpath("names.txt").read_text().splitlines()
end_tok = "."
chars = [end_tok] + sorted(list(set("".join(words))))
char_to_index = {char: index for index, char in enumerate(chars)}
n_chars = len(chars)

block_size = 3
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + end_tok:
        ix = char_to_index[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(chars[i] for i in context), "->", ch)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)
num = X.nelement()

In [160]:
n_embeddings_dims = 2

C = torch.randn(27, n_embeddings_dims)
layer1_size = 100
W1 = torch.randn((n_embeddings_dims * block_size, layer1_size))
b1 = torch.randn(layer1_size)
layer2_size = 27
W2 = torch.randn((layer1_size, layer2_size))
b2 = torch.randn(layer2_size)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [161]:
steps = 10000
lre = torch.linspace(-3, 0, steps)
lrs = 10**lre

In [ ]:
batch_size = 128
lri = []
lossi = []

for i in range(steps):
    # Get minibatch
    ix = torch.randint(0, X.shape[0], (batch_size,))

    # Forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, n_embeddings_dims * block_size) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    print(loss.item())

    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    lr = .01
    for p in parameters:
        p.data += -1 * p.grad

    # lri.append(lre[i])
    # lossi.append(loss.item())

In [179]:
# Eval
emb = C[X]
h = torch.tanh(emb.view(-1, n_embeddings_dims * block_size) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.346143960952759